# Description 
- Nội dung: dữ liệu ghi lại thông tin các vụ phóng tên lửa từ năm 1957
- Nguồn: https://www.kaggle.com/agirlcoding/all-space-missions-from-1957
- Cách thức dữ liệu được lấy: dữ liệu được scrap từ trang https://nextspaceflight.com/launches/past/?page=1

# Import libraries

In [281]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

# Import file

In [282]:
spaceLaunch_df = pd.read_csv("Space_Corrected.csv")
spaceLaunch_df.head()

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


# Explore

In [283]:
#Dữ liệu gồm 4324 dòng và 9 cột
spaceLaunch_df.shape

(4324, 9)

In [284]:
# 9 cột tương ứng là 'Unnamed: 0','Unnamed: 0.1','Company Name','Location','Datum','Detail','Status Rocket',' Rocket','Status Mission'
spaceLaunch_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Company Name', 'Location', 'Datum',
       'Detail', 'Status Rocket', ' Rocket', 'Status Mission'],
      dtype='object')

In [285]:
spaceLaunch_df.dtypes

Unnamed: 0         int64
Unnamed: 0.1       int64
Company Name      object
Location          object
Datum             object
Detail            object
Status Rocket     object
 Rocket           object
Status Mission    object
dtype: object

## Ý nghĩa các cột dữ liệu
- Unnamed: 0,Unnamed: 0.1: (int) trường index
- Company Name: (int) tên của công ty thực hiện vụ phóng
- Location: (object) địa điểm xảy ra vụ phóng
- Datum: (object) Thời điểm vụ phóng xả ra
- Detail: (object) Phân lớp/Tên của loại tên lửa
- Status Rocket: (object) Tình trạng của tên lửa (đang còn sử dụng/không còn sử dụng)
- Rocket: (object) Chi phí cho tên lửa
- Status Mission: (object) Kết quả của vụ phóng
    

In [286]:
# Xác định các kiểu dữ liệu có thể có của object
def open_object_dtype(s):
    dtypes = set()
    
    # YOUR CODE HERE
    for i in range(len(s)):
        dtypes.add(type(s.iloc[i]))
    
    return dtypes

In [287]:
# Company Name: (object) tên của công ty thực hiện vụ phóng (56 công ty)
print(len(spaceLaunch_df["Company Name"].unique()))
open_object_dtype(spaceLaunch_df["Location"])

56


{str}

In [288]:
# Location: (object) địa điểm xảy ra vụ phóng (137 địa điểm)
print(len(spaceLaunch_df["Location"].unique()))
open_object_dtype(spaceLaunch_df["Location"])

137


{str}

In [289]:
# Datum: (object) Thời điểm vụ phóng xả ra 
open_object_dtype(spaceLaunch_df["Datum"])

{str}

In [290]:
# Detail: (object) Phân lớp/Tên của loại tên lửa (4278 giá trị phân biệt)
print(len(spaceLaunch_df["Detail"].unique()))
open_object_dtype(spaceLaunch_df["Detail"])

4278


{str}

In [291]:
# Status Rocket: (object) Tình trạng của tên lửa (đang còn sử dụng/không còn sử dụng)
print(len(spaceLaunch_df["Status Rocket"].unique()))
open_object_dtype(spaceLaunch_df["Status Rocket"])

2


{str}

In [292]:
# Rocket: (object) Chi phí cho tên lửa
open_object_dtype(spaceLaunch_df[" Rocket"])

{float, str}

In [293]:
# Status Mission: (object) Kết quả của vụ phóng (Success/Failure/Prelaunch Failure/Partial Failure)
spaceLaunch_df["Status Mission"].unique()
open_object_dtype(spaceLaunch_df["Status Mission"])

{str}

# Data Preprocessing

In [294]:
#Bỏ đi 2 cột Unnamed: 0 và Unnamed: 0.1 vì không cần thiết
del spaceLaunch_df["Unnamed: 0"]
del spaceLaunch_df["Unnamed: 0.1"]

In [295]:
#Convert cột rocket sang kiểu Float
spaceLaunch_df[" Rocket"] = spaceLaunch_df[" Rocket"].astype(str)
for i in range(len(spaceLaunch_df[" Rocket"])):
    spaceLaunch_df[" Rocket"][i] = spaceLaunch_df[" Rocket"][i].strip(", ")
    spaceLaunch_df[" Rocket"][i] = spaceLaunch_df[" Rocket"][i].replace(",","")
spaceLaunch_df[" Rocket"] = spaceLaunch_df[" Rocket"].astype(float)

In [296]:
#Tên cột Rocket bị dư một dấu cách phía trước, không để ý có thể gây lỗi nên cần được bỏ dấu cách đi
spaceLaunch_df = spaceLaunch_df.rename(columns= {" Rocket" : "Rocket"})

#Cột Datum là kiểu dữ liệu object cần được chuyển sang dạng datetime
spaceLaunch_df["Datum"] = pd.to_datetime(spaceLaunch_df["Datum"])

#Tạo thêm cột month, year
spaceLaunch_df["year"] = spaceLaunch_df["Datum"].apply(lambda datetime: datetime.year)
spaceLaunch_df["month"] = spaceLaunch_df["Datum"].apply(lambda datetime: datetime.month)

#Tạo thêm cột country
spaceLaunch_df["country"] = spaceLaunch_df["Location"].apply(lambda country: country.split(", ")[-1])

In [297]:
#Các nhiệm vụ phóng được thực hiện tại Kazakhstan đều được thực hiện bởi 
#Nga nên ta cần đổi giá trị của cột country mà là Kazakhstan thành Russia mà trước năm 1991 là Liên Xô (USSR)
for i in range(len(spaceLaunch_df["country"])):
    if (spaceLaunch_df["country"].loc[i] == "Kazakhstan"):
        if(spaceLaunch_df["year"].loc[i] < 1991):
            spaceLaunch_df["country"].loc[i] = "USSR"
        else:
            spaceLaunch_df["country"].loc[i] = "Russia"
    # Một vài đại điểm phóng tại Nga trước năm 1991 vẫn lưu là Nga nên ta đổi thành USSR
    if (spaceLaunch_df["country"].loc[i] == "Russia" and spaceLaunch_df["year"].loc[i] < 1991):
        spaceLaunch_df["country"].loc[i] = "USSR"

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Dữ liệu sau tiền xử lý

In [298]:
spaceLaunch_df

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission,year,month,country
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,2020,8,USA
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,2020,8,China
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,2020,8,USA
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,2020,7,Russia
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success,2020,7,USA
...,...,...,...,...,...,...,...,...,...,...
4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00+00:00,Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,1958,2,USA
4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00+00:00,Juno I | Explorer 1,StatusRetired,NaN,Success,1958,2,USA
4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00+00:00,Vanguard | Vanguard TV3,StatusRetired,NaN,Failure,1957,12,USA
4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00+00:00,Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success,1957,11,USSR


In [299]:
spaceLaunch_df.dtypes

Company Name       object
Location           object
Datum              object
Detail             object
Status Rocket      object
Rocket            float64
Status Mission     object
year                int64
month               int64
country            object
dtype: object

# Analysis

## Question 1

In [ ]:
space_company_5_df = spaceLaunch_df[spaceLaunch_df["Year"] > 2015]
space_company_5_df = space_company_5_df[~np.isnan(space_company_5_df["Rocket"])]

company_success_launch_df = space_company_5_df[space_company_5_df["Status Mission"] == "Success"].groupby(["Year","Company Name"])["Status Mission"].count().sort_values(ascending = False) 
company_launch_df = space_company_5_df.groupby("Year","Company Name")["Status Mission"].count().sort_values(ascending = False)

company_success_launch_ratio_df = company_success_launch_df.div(company_launch_df).reset_index()
company_success_launch_ratio_df = company_success_launch_ratio_df.rename(columns= {"Status Mission" : "Success Ratio"})
company_success_launch_ratio_df = company_success_launch_ratio_df.fillna(0)#.sort_values(by = "Success Ratio")
company_success_launch_ratio_df["Launch"] = np.zeros(len(company_success_launch_ratio_df["Company Name"]))
company_success_launch_ratio_df["Success"] = np.zeros(len(company_success_launch_ratio_df["Company Name"]))

for i in range(len(company_success_launch_ratio_df["Company Name"])):
    company_success_launch_ratio_df.loc[i,"Launch"] = company_launch_df[company_success_launch_ratio_df["Company Name"][i]]
    if(company_success_launch_ratio_df["Success Ratio"][i] != 0):
        company_success_launch_ratio_df.loc[i,"Success"] = company_success_launch_df[company_success_launch_ratio_df["Company Name"][i]]
    else:
        company_success_launch_ratio_df.loc[i,"Success"] = 0
company_success_launch_ratio_df.sort_values(by = "Launch").tail(10)


In [ ]:
space_company_5_df = spaceLaunch_df[spaceLaunch_df["year"] > 2015]
space_company_5_df = space_company_5_df[~np.isnan(space_company_5_df["Rocket"])]

company_success_launch_df = space_company_5_df[space_company_5_df["Status Mission"] == "Success"].groupby(["year","Company Name"])["Status Mission"].count()
company_launch_df = space_company_5_df.groupby(["year","Company Name"])["Status Mission"].count()
company_launch_df

## Question 2: Số lượng nhiệm vụ phóng qua mỗi năm

In [302]:
launchCountry = spaceLaunch_df.groupby(["year","country"])["Status Mission"].count().reset_index().sort_values(["year","Status Mission"],ascending = False)
launchCountry = pd.concat([group[1].head(1) for group in launchCountry.groupby(['year'])])

fig = px.bar(launchCountry, x="year", y="Status Mission", color='country')
fig.show()

Từ biểu đồ này ta có thể thấy trước 2017 gần như là Mỹ và Nga thay nhau đứng đầu về số lượng nhiệm vụ ra ngoài không gian, nhưng 3 năm gần đây thì Trung Quốc lại nổi lên một cách mạnh mẽ vượt qua 2 cường quốc kia dù là nước đi sau trong cuộc đua không gian này, liệu ta có thể hiểu thêm bằng cách phân tích cuộc đua của các công ty qua các năm?

In [319]:
spaceLaunch_China_df = spaceLaunch_df[spaceLaunch_df["country"] == "China"]
spaceLaunch_China_df.groupby(["Company Name"])["Status Mission"].count()

Company Name
CASC         250
CASIC          5
ExPace        10
Landspace      1
OneSpace       1
i-Space        1
Name: Status Mission, dtype: int64

In [ ]:
CASC là một công ty nhà nước về khoa học hàng không vũ trụ, ta xem xét thử sự phát triển của công ty này

In [322]:
CASC = spaceLaunch_China_df[spaceLaunch_China_df["Company Name"] == "CASC"]
# launchCompany_China = pd.concat([group[1].head(1) for group in launchCompany_China.groupby(['year'])])
CASC = CASC.groupby("year")["Status Mission"].count()
CASC

year
1969     1
1970     1
1971     1
1972     1
1973     1
1974     2
1975     3
1976     3
1977     1
1978     2
1979     1
1981     1
1982     1
1983     1
1984     3
1985     1
1986     2
1987     2
1988     4
1990     5
1991     1
1992     4
1993     1
1994     5
1995     3
1996     4
1997     6
1998     6
1999     4
2000     4
2001     1
2002     4
2003     6
2004     8
2005     5
2006     6
2007    10
2008     9
2011     2
2012     1
2013     1
2015     2
2016    22
2017    16
2018    37
2019    26
2020    19
Name: Status Mission, dtype: int64

In [305]:
plt.xlabel("year")
plt.ylabel("Missions")
plt.plot(launchCompany[launchCompany["Company Name"] == "CASC"],marker = "o", c = "red", label = "CASC")
plt.plot(launchCompany[launchCompany["Company Name"] == "SpaceX",ls = '-', marker = "o", c = "green", label = "SpaceX")
plt.plot(launchCompany[launchCompany["Company Name"] == ,ls = '-.', marker = "o", c = "blue",label = "2012")
plt.legend()

,year,Company Name,Status Mission
2,2018,CASC,37
13,2018,SpaceX,21
0,2018,Arianespace,11
18,2019,CASC,27
29,2019,Roscosmos,13
30,2019,SpaceX,13
35,2020,CASC,19
45,2020,SpaceX,14
34,2020,Arianespace,4


## Question 3: Liệu ngành công nghiệp dịch vụ du lịch trong vũ trụ có khả thi?